In [3]:
import pandas as pd
import numpy as np
import os
import sklearn.datasets
import sklearn.metrics
import docx2txt
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tokenize import word_tokenize, sent_tokenize
import random

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
s_words= set(stopwords.words('english'))
s_words1={'.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'}
s_words_more={"sender:","date:" ,"like:","|>","the"
                ,"subject:", 'I',"from:",'>',"newsgroups:", "message-id:", "path:","apr","lines:", '|','-','--',"would", "re:", "references:",
                "1993","article",'*',"xref", '1','2',"make", "good", "may", "even","organization:","gmt","writes:","xref"}


In [8]:
dir_path = "20_newsgroups_1"   # Path of the directory
file_path=[]    #  Counting the no. of classes and storing them in  a array 
for files in os.listdir("20_newsgroups_1"):
    dict[files]= "None"
    file_path.append(files)
# print(file_path)
    

In [9]:
files = []      # Iterating all the folders and storing them in a array named as file
for dirname, dirnames, filenames in os.walk(dir_path):
    files += [os.path.join(dirname, filename) for filename in filenames]
# print(files)




In [10]:
# Shuffling all the files
random.shuffle(files)


In [11]:
# Splitting into data_train and data_test after shuffling
data_train=[]
data_test=[]
for i in range(17000):
    data_train.append(files[i])
    
for i in range(17000, 19997):
    data_test.append(files[i])
# len(data_train)

In [14]:

complete_words={}     ##dictionary to store the frequency of all the words in data_train
for file in data_train:
    f=open(file,'r')
    for words in f.read().split():
        words=words.lower()
        if words not in s_words and words not in s_words1 and words not in s_words_more:  #if word not a stopword 
                
                if words in complete_words:
                    complete_words[words] += 1
                else:
                    complete_words[words] = 1


In [15]:
fdist = FreqDist(complete_words).most_common(5000)  #picking up top 5000 words from the dictionary
dict_fdist={}
for i in range(len(fdist)):
    dict_fdist[fdist[i][0]]= fdist[i][1]       #creating dictionary for top 5000 words
        
# dict_fdist  

In [16]:
data_train_dictionary={}     # dictionary for training data_elements
for file in data_train:                 #data_train is the list of files to be trained 
    path_list = file.split(os.sep)   #for separting '20_newsgroups_1\\rec.sport.baseball\\104828' into three parts
    class_label= path_list[1]              #after separationg eg. rec.sport.baseball is the 1 index and is named class_label
    if class_label in data_train_dictionary:    #checking if following class is in dictionary or not
        values= data_train_dictionary[class_label]
        f=open(file,'r')
        for words in f.read().split():
            words=words.lower()
            if words in dict_fdist:     #if word available in top 2000 words, dict_fdist is dictionary containing top 2000 words
                if words in values:    # if word already available in available class dictionary
                    word_count = data_train_dictionary[class_label][words]
                    data_train_dictionary[class_label][words]=word_count+1
                else:
                    data_train_dictionary[class_label][words]=1
    else:
        data_train_dictionary[class_label]={}
        dict_word_freq={}
        f=open(file,'r')
        for words in f.read().split():
            words=words.lower()
            if words in dict_fdist:
                if words in dict_word_freq:
                    dict_word_freq[words] += 1
                else:
                    dict_word_freq[words] = 1
        data_train_dictionary[class_label]= dict_word_freq 

# print(data_train_dictionary)
   

In [18]:
total_sum=0
dict_sum={}

for key in data_train_dictionary.keys():
    values= data_train_dictionary[key]
    sum_class=0
    for value in values:
#         print(data_train_dictionary[key][value], end=" ")
        sum_class+= data_train_dictionary[key][value]
        total_sum+= data_train_dictionary[key][value]
    dict_sum[key]= sum_class

        
        

In [20]:
def probability(dictionary, x , current_class):
    
    sum_current_class_entries= dict_sum[current_class]
    prob= np.log(1)-np.log(20)
    f=open(x,'r')
    for words in f.read().split():
        words=words.lower()
        values= dictionary[current_class]
        if(words in values):
             prob+= np.log(dictionary[current_class][words] +1 )- np.log(sum_current_class_entries+len(dict_fdist))
        else:
             prob+= np.log(1)- np.log(len(dict_fdist))
        
    return prob
        
   


In [21]:
def predictSinglePoint(dictionary, x):
    
    classes= dictionary.keys()
    best_prob= -1000000
    best_class = -10
    first_run = True
    for current_class in classes:
        
        p_current_class = probability(dictionary, x, current_class)
#         print(p_current_class, end=" ")
        if( first_run or p_current_class> best_prob):
            best_prob= p_current_class
            best_class = current_class
        first_run = False
#         print()
        
    return best_class

In [22]:
def test_dictionary(dictionary, x):
    dict_test_freq={}
    f=open(x,'r')
    for words in f.read().split():
            words=words.lower()
#             print(words)
            if words in dict_fdist:
#                 print(" i am in")
#                 print(words,  class_label)
                if words in dict_test_freq:
                    dict_test_freq[words] += 1
                else:
                    dict_test_freq[words] = 1
    return dict_test_freq

In [23]:
def predict(dictionary, X_test):
    
    y_pred=[]
    Y_test_initial=[]
    for x in X_test:
        path_list = x.split(os.sep)
        class_label= path_list[1]
        Y_test_initial.append(class_label)
#         print(class_label,  x)
#         dictionary_test= test_dictionary(dictionary, x)
#         print(dictionary_test)
        x_class = predictSinglePoint(dictionary, x)
        y_pred.append(x_class)
    return y_pred,Y_test_initial

In [24]:

Y_predict,Y_test_in= predict(data_train_dictionary, data_test)
print(Y_predict)
    

['talk.religion.misc', 'comp.sys.mac.hardware', 'rec.sport.hockey', 'sci.space', 'comp.sys.mac.hardware', 'comp.os.ms-windows.misc', 'rec.sport.baseball', 'comp.windows.x', 'comp.sys.ibm.pc.hardware', 'comp.sys.ibm.pc.hardware', 'rec.autos', 'comp.sys.ibm.pc.hardware', 'talk.religion.misc', 'alt.atheism', 'sci.space', 'sci.crypt', 'comp.windows.x', 'rec.autos', 'comp.sys.ibm.pc.hardware', 'rec.motorcycles', 'talk.politics.mideast', 'rec.motorcycles', 'soc.religion.christian', 'talk.politics.misc', 'soc.religion.christian', 'comp.windows.x', 'alt.atheism', 'sci.electronics', 'misc.forsale', 'comp.sys.ibm.pc.hardware', 'talk.politics.mideast', 'sci.med', 'comp.graphics', 'rec.autos', 'misc.forsale', 'rec.motorcycles', 'rec.sport.baseball', 'soc.religion.christian', 'comp.sys.mac.hardware', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'sci.electronics', 'sci.crypt', 'comp.sys.mac.hardware', 'alt.atheism', 'rec.sport.baseball', 'talk.religion.misc', 'rec.sport.baseball', 'c

In [25]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_test_in,Y_predict))
print(confusion_matrix(Y_test_in,Y_predict))

                          precision    recall  f1-score   support

             alt.atheism       0.70      0.79      0.74       151
           comp.graphics       0.76      0.76      0.76       139
 comp.os.ms-windows.misc       0.81      0.77      0.79       142
comp.sys.ibm.pc.hardware       0.75      0.81      0.78       150
   comp.sys.mac.hardware       0.79      0.86      0.83       153
          comp.windows.x       0.90      0.82      0.86       159
            misc.forsale       0.76      0.85      0.81       143
               rec.autos       0.78      0.86      0.81       154
         rec.motorcycles       0.80      0.95      0.87       155
      rec.sport.baseball       0.89      0.91      0.90       149
        rec.sport.hockey       0.93      0.92      0.92       151
               sci.crypt       0.94      0.86      0.90       146
         sci.electronics       0.73      0.81      0.77       134
                 sci.med       0.91      0.81      0.85       156
         